# The `instrument` package for Bluesky Data Acquisition

From *APS Python Training for Bluesky Data Acquisition*.

**Objective**

In this notebook, we describe the `instrument` package for a simulated X-ray instrument at a user facility such as the Advanced Photon Source.  The goal is to use EPICS as much as possible to provide the control system features and to use Bluesky (providing the data acquisition framework) as a thin layer on top of EPICS.

**Contents**

* [Start the `instrument` package](#start-the-instrument-package)
* [Description](#Description)
  * [`adsimdet`](#adsimdet)
  * [`scaler1`](#scaler1)
  * [`temperature`](#temperature)
  * [motors](#motors)
  * [`noisy`](#noisy)
  * [plans](#plans)
* [Log files](#log-files)
* [SPEC data files](#spec-data-files)
* [User code file](#user-code-file)
* [`databroker`](#databroker)

**Overview**

This simulated instrument is provided using [docker](https://www.docker.com/) images
for [EPICS base](https://www.aps.anl.gov/epics), the [synApps](https://www.aps.anl.gov/BCDA/synApps) [xxx](https://github.com/epics-modules/xxx) module, and [EPICS area detector](https://areadetector.github.io/master/index.html) [ADSimDetector](https://areadetector.github.io/master/ADSimDetector/simDetector.html?highlight=adsimdetector).  The images are based on these software versions:

* Ubuntu 20.04
* EPICS base 7.0.5
* synApps 6.2
* area detector 3.10

Two EPICS IOCs are provided:

prefix | description | docker image | documentation
:--- | :--- | :--- | :---
`ad:` | area detector IOC | [`prjemian/custom-synapps-6.2-ad-3.10`](https://hub.docker.com/r/prjemian/custom-synapps-6.2-ad-3.10) | https://github.com/prjemian/epics-docker/tree/main/v1.1/n6_custom_areaDetector
`gp:` | general purpose IOC | [`prjemian/prjemian/custom-synapps-6.2`](https://hub.docker.com/r/prjemian/custom-synapps-6.2) | https://github.com/prjemian/epics-docker/tree/main/v1.1/n5_custom_synApps

## Start the `instrument` package

In [2]:
import os, sys
sys.path.append(os.path.join(".", "bluesky"))
from instrument.collection import *

I Tue-21:52:22 - ############################################################ startup
I Tue-21:52:22 - logging started
I Tue-21:52:22 - logging level = 10
I Tue-21:52:22 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/collection.py
I Tue-21:52:22 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Tue-21:52:23 - #### Bluesky Framework ####
I Tue-21:52:23 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/framework/check_python.py
I Tue-21:52:23 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/framework/check_bluesky.py
I Tue-21:52:24 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/framework/initialize.py
I Tue-21:52:27 - using databroker catalog 'training'
I Tue-21:52:27 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/framework/metadata.py
I Tue-21:52:27 - #### Devices ####
I Tue-21:52:27 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/devices/area_detector.py
I Tue-21:52:27 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/devices/calculation_records.py
I Tue-21:52:31 - Priming adsimdet_hdf1 ...
I Tue-21:52:34 - Enabling adsimdet_image plugin ...
I Tue-21:52:35 - /home/

## Description

Might be a good idea to know now what this instrument package provides.  

Notably, the table includes:

ophyd name(s) | Description | label(s)
:--- | :---| :---
`adsimdet` | simulated EPICS area detector | `area_detector`
`calcs` & `calcouts` | calculation support |
`gp_stats` | details about the general purpose IOC |
`I0`, `diode`, ... | named scaler channels | `counter`
`m1` .. `m16` | 16 simulated EPICS motors | `motor`
`noisy` | simulated diffraction peak |
`scaler1` | simulated 16-channel EPICS scaler | `scalers`
`shutter` | simulated shutter | 
`temperature` | simulated temperature controller |

### `adsimdet`

The EPICS Area Detector ADSimDetector is included with this instrument as `adsimdet`.  The detector is a monochrome 1k x 1k frame.  The image is written to an HDF5 file and made available to the [`databroker`](#databroker) via a shared volume from the docker container.  `adsimdet` can be used as a detector.

The image is a simulated diffraction spot with center randomly-chosen between 100..900 on both axes.  The width is also random as is the peak intensity and noise level.  Furthermore, to simulate realistic conditions, the center position of both axes is adjusted by a few pixels using a pair of *swait* records (`gp:userCalc9` & `gp:userCalc10`) updating from random numbers.

In [3]:
adsimdet.summary()

data keys (* hints)
-------------------

read attrs
----------
hdf1                 MyHDF5Plugin        ('adsimdet_hdf1')

config keys
-----------
adsimdet_cam_acquire_period
adsimdet_cam_acquire_time
adsimdet_cam_image_mode
adsimdet_cam_manufacturer
adsimdet_cam_model
adsimdet_cam_num_exposures
adsimdet_cam_num_images
adsimdet_cam_trigger_mode

configuration attrs
-------------------
cam                  MyFixedCam          ('adsimdet_cam')
cam.acquire_period   EpicsSignalWithRBV  ('adsimdet_cam_acquire_period')
cam.acquire_time     EpicsSignalWithRBV  ('adsimdet_cam_acquire_time')
cam.image_mode       EpicsSignalWithRBV  ('adsimdet_cam_image_mode')
cam.manufacturer     EpicsSignalRO       ('adsimdet_cam_manufacturer')
cam.model            EpicsSignalRO       ('adsimdet_cam_model')
cam.num_exposures    EpicsSignalWithRBV  ('adsimdet_cam_num_exposures')
cam.num_images       EpicsSignalWithRBV  ('adsimdet_cam_num_images')
cam.trigger_mode     EpicsSignalWithRBV  ('adsimdet_cam_trigger_m

### `scaler1`

A simulated 16-channel scaler is configured with several channels of pulse counters, typical of a synchrotron beam line.  `scaler1` can be used as a detector.

In [4]:
scaler1.summary()

data keys (* hints)
-------------------
*I0
*diode
 scaler1_time
*scint
*timebase

read attrs
----------
channels             Channels            ('scaler1_channels')
channels.chan01      ScalerChannel       ('scaler1_channels_chan01')
channels.chan01.s    EpicsSignalRO       ('timebase')
channels.chan02      ScalerChannel       ('scaler1_channels_chan02')
channels.chan02.s    EpicsSignalRO       ('I0')
channels.chan03      ScalerChannel       ('scaler1_channels_chan03')
channels.chan03.s    EpicsSignalRO       ('scint')
channels.chan04      ScalerChannel       ('scaler1_channels_chan04')
channels.chan04.s    EpicsSignalRO       ('diode')
time                 EpicsSignal         ('scaler1_time')

config keys
-----------
scaler1_auto_count_delay
scaler1_auto_count_time
scaler1_channels_chan01_chname
scaler1_channels_chan01_gate
scaler1_channels_chan01_preset
scaler1_channels_chan02_chname
scaler1_channels_chan02_gate
scaler1_channels_chan02_preset
scaler1_channels_chan03_chname
scaler1_

The timebase is always shown.  The additional channels (using names from the scaler GUI screen) are shown output of `listdevice(scaler1)` below.

Note that in the instrument package, the channel names were assigned on startup.  An operating beam line would not do define the names here but instead, let the instrument team name these channels in the GUI screen at the time the cables from the pulse detectors are connected physically to the scaler.

Count the scaler from the command line:

In [5]:
%ct scalers

[This data will not be saved. Use the RunEngine to collect data.]
timebase                       11000000.0
I0                             5.0
scint                          5.0
diode                          4.0
scaler1_time                   1.1


NOTE: the actual counts are random numbers.  Also, the EPICS soft scaler seems to add 0.1s to the counting time.  Might be a bug with the EPICS scaler code.

In [6]:
listdevice(scaler1)

============ ========== ==========================
name         value      timestamp                 
============ ========== ==========================
timebase     11000000.0 2021-08-31 21:52:40.073369
I0           5.0        2021-08-31 21:52:40.073369
scint        5.0        2021-08-31 21:52:40.073369
diode        4.0        2021-08-31 21:52:40.073369
scaler1_time 1.1        1989-12-31 18:00:00       
============ ========== ==========================



As an added convenience, shortcut names for these channels were also assigned to local Python symbols (since the object path to the channels is easy to forget) in `instrument/devices/scaler.py` with these lines:

```python
timebase = scaler1.channels.chan01.s
I0 = scaler1.channels.chan02.s
scint = scaler1.channels.chan03.s
diode = scaler1.channels.chan04.s
```

This command: `scaler1.select_channels()` configures `scaler` to use just the channels with non-empty names.  To limit data collection to a subset of these channels, name the channels in a list argument, such as: `scaler1.select_channels(['I0', 'diode'])`

### `temperature`

The temperature simulator, implemented in `gp:userCalc8` works like a temperature controller, with setpoint and readback values.    `temperature` can be used as a detector **and** as a positioner (like a motor).

Additional controls for noise, update interval, maximum change per update, in-position tolerance, and done are provided, either through EPICS or in Python.  Once a new setpoint is entered, the readback will progress towards it, limited by the max change and the update interval.  Once at the setpoint, noise continues to be added to the temperature to provide realism.

In [7]:
temperature.summary()
listdevice(temperature)

data keys (* hints)
-------------------
*temperature
 temperature_calculation
 temperature_description
 temperature_done
 temperature_max_change
 temperature_noise
 temperature_previous_value_pv
 temperature_scanning_rate
 temperature_setpoint
 temperature_tolerance

read attrs
----------
setpoint             EpicsSignal         ('temperature_setpoint')
readback             EpicsSignal         ('temperature')
done                 Signal              ('temperature_done')
calculation          EpicsSignal         ('temperature_calculation')
description          EpicsSignal         ('temperature_description')
max_change           EpicsSignal         ('temperature_max_change')
noise                EpicsSignal         ('temperature_noise')
previous_value_pv    EpicsSignal         ('temperature_previous_value_pv')
scanning_rate        EpicsSignal         ('temperature_scanning_rate')
tolerance            EpicsSignal         ('temperature_tolerance')

config keys
-----------

configuration att

### motors

There are 16 soft motor channels: `gp:m1` .. `gp:m16`.  Any motor can be used as a detector **and** as a positioner.

The first motor, `gp:m1`, is used to compute a simulated 1-D diffraction peak ([`noisy`](#noisy)) using `gp:userCalc1`.

To make it easier to change the motor step size, a custom `MyEpicsMotor` class was made, subclassing from `ophyd.EpicsMotor` and adding a component to access the motor's `.SREV` (steps/revolution) field.  The default motor step size is 0.01 (with SREV=200).  For only `m1`, the step size has been changed to 0.001 by setting SREV=2000.

NOTE: Changing SREV (in this simulator* seems to affect the soft motor's simulated hardware limit.  Be careful if you adjust SREV to higher numbers (smaller step sizes) that the expected range of motion remains sufficient for your measurements.  Again, another thing to change in the EPICS support.

In [8]:
m1.summary()
listdevice(m1)

data keys (* hints)
-------------------
*m1
 m1_user_setpoint

read attrs
----------
user_readback        EpicsSignalRO       ('m1')
user_setpoint        EpicsSignal         ('m1_user_setpoint')

config keys
-----------
m1_acceleration
m1_motor_egu
m1_user_offset
m1_user_offset_dir
m1_velocity

configuration attrs
-------------------
user_offset          EpicsSignal         ('m1_user_offset')
user_offset_dir      EpicsSignal         ('m1_user_offset_dir')
velocity             EpicsSignal         ('m1_velocity')
acceleration         EpicsSignal         ('m1_acceleration')
motor_egu            EpicsSignal         ('m1_motor_egu')

unused attrs
------------
offset_freeze_switch EpicsSignal         ('m1_offset_freeze_switch')
set_use_switch       EpicsSignal         ('m1_set_use_switch')
motor_is_moving      EpicsSignalRO       ('m1_motor_is_moving')
motor_done_move      EpicsSignalRO       ('m1_motor_done_move')
high_limit_switch    EpicsSignalRO       ('m1_high_limit_switch')
low_limit_s

### `noisy`

A simulated diffraction peak is computed in 1-D using a Lorentzian function in EPICS PV `gp:userCalc1`.  The simulation uses the position of motor `gp:m1` and random choices for center, width, noise, and peak intensity (scale factor).  `noisy` can be used as a detector.

NOTE: `noisy` is an `ophyd.EpicsSignal` (variant), thus it lacks the `.summary()` method that `ophyd.Device` objects have.

In [9]:
listdevice(noisy)

===== ================= ==========================
name  value             timestamp                 
===== ================= ==========================
noisy 4485.008066942357 2021-08-31 21:52:35.845222
===== ================= ==========================



### plans

Several examples of user plan are provided in file `instrument/plans/peak_finder_example.py`.  These plans are described and used in the notebook [Lineup a 1-D peak](https://nbviewer.jupyter.org/github/BCDA-APS/bluesky_instrument_training/blob/main/lineup_1d_peak.ipynb).

## Log files

For diagnostic and general support, log files are created to record activity.  In the working directory, the log files are written to a `./.logs` subdirectory.

There are two kinds of file, one that records user commands and the python
result, the other records items sent to the
[customized](https://github.com/prjemian/stdlogpj#example-directing-logs-to-a-specific-directory)
Python [logging](https://docs.python.org/3/library/logging.html) package.

In the IPython session, use the `!` to run a linux command:

In [10]:
!ls -lAFgh .logs

total 1.1M
-rw-rw-r-- 1 mintadmin  741 Aug 31 21:56 ipython_console.log
-rw-rw-r-- 1 mintadmin   20 Aug 25 17:03 ipython_console.log.001~
-rw-rw-r-- 1 mintadmin  78K Aug 21 22:52 ipython_console.log.002~
-rw-rw-r-- 1 mintadmin 5.6K Aug 21 21:01 ipython_console.log.003~
-rw-rw-r-- 1 mintadmin 5.9K Aug 20 23:09 ipython_console.log.004~
-rw-rw-r-- 1 mintadmin 5.3K Aug 20 23:08 ipython_console.log.005~
-rw-rw-r-- 1 mintadmin 1.9K Aug 20 23:07 ipython_console.log.006~
-rw-rw-r-- 1 mintadmin 2.8K Aug 20 23:07 ipython_console.log.007~
-rw-rw-r-- 1 mintadmin 8.8K Aug 20 22:54 ipython_console.log.008~
-rw-rw-r-- 1 mintadmin 3.0K Aug 20 22:47 ipython_console.log.009~
-rw-rw-r-- 1 mintadmin  17K Aug 20 22:43 ipython_console.log.010~
-rw-rw-r-- 1 mintadmin 3.1K Aug 20 22:35 ipython_console.log.011~
-rw-rw-r-- 1 mintadmin 3.9K Jul 23 21:45 ipython_console.log.012~
-rw-rw-r-- 1 mintadmin  678 Jul 23 21:44 ipython_console.log.013~
-rw-rw-r-- 1 mintadmin  11K Jul 23 21:04 ipython_console.log.014~
-rw-

Let's take a look at a few lines of each type of file, to get a feel for the information logged.

The `ipython_console.log` file is created for every session (every time IPython is started or every time the Jupyter kernel is started).  It records the commands that were entered and the output, if any, from each command.  It does not record anything that was sent to the console by `print()` statements.  The older log files are numbered, higher number is older (each new session shifts these numbers by 1).

In [11]:
!head .logs/ipython_console.log

# IPython log file

## Start the `instrument` package
# Tue, 31 Aug 2021 21:52:38
adsimdet.summary()
# Tue, 31 Aug 2021 21:52:38
scaler1.summary()
# Tue, 31 Aug 2021 21:52:38
get_ipython().run_line_magic('ct', 'scalers')
# Tue, 31 Aug 2021 21:52:45


The `ipython_logger.log` file(s) contain the output from calls to the `logger`.  The files are appended with new logger reports until the file reaches ca. 1 MB.  (Larger files are slow to append.)  Then the file is given a number and a new logger file is created.  Higher numbers are older.  At most, 9 numbered files are retained (to avoid filling disk storage with unnecessary diagnostics).

A single *logger* file may contain reports from several sessions.  The third piece of information is the `pid` (process identifier) of the session.  The PID is assigned by the operating system when the session is started.

The logger file contains a full report while the report's representation is more terse.  (Users do not always want the full details.  Just remember they are available in the logger file.)  Here's the last few logger lines as shown on the console:

    I Wed-00:24:52 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/temperature_signal.py
    I Wed-00:24:52 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/plans/peak_finder_example.py
    I Wed-00:24:52 - Startup is complete.

Those same lines appear in the logger file as:

    |2021-02-24 00:24:52.091|INFO|25986|bluesky-session|temperature_signal|11|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/temperature_signal.py
    |2021-02-24 00:24:52.165|INFO|25986|bluesky-session|peak_finder_example|15|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/plans/peak_finder_example.py
    |2021-02-24 00:24:52.179|INFO|25986|bluesky-session|collection|23|MainThread| - Startup is complete.


In [12]:
!tail .logs/ipython_logger.log

|2021-08-31 21:52:37.970|INFO|20507|bluesky-session|spec_data_file_writer|13|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/./bluesky/instrument/callbacks/spec_data_file_writer.py
|2021-08-31 21:52:37.971|INFO|20507|bluesky-session|spec_data_file_writer|29|MainThread| - writing to SPEC file: /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/20210831-215237.dat
|2021-08-31 21:52:37.971|INFO|20507|bluesky-session|spec_data_file_writer|30|MainThread| -    >>>>   Using default SPEC file name   <<<<
|2021-08-31 21:52:37.972|INFO|20507|bluesky-session|spec_data_file_writer|31|MainThread| -    file will be created when bluesky ends its next scan
|2021-08-31 21:52:37.972|INFO|20507|bluesky-session|spec_data_file_writer|32|MainThread| -    to change SPEC file, use command:   newSpecFile('title')
|2021-08-31 21:52:37.973|INFO|20507|bluesky-session|collection|20|MainThread| - #### Plans ####
|2021-08-31 21:52:37.975|INFO|20507|bluesky-session|peak_finder_exa

## SPEC data files

A [SPEC](https://certif.com) data file records a copy of the scan data (area detector images are not stored in SPEC files).  By default, the file is created in the present working directory using a name constructed from the date and time with a `.dat` file extension.  Unlike SPEC, new data is written *at the end of a scan*.  It is also possible to use the [`SpecWriterCallback`](https://apstools.readthedocs.io/en/latest/source/_filewriters.html#apstools.filewriters.SpecWriterCallback) to write data extracted from the database after the experiment is done.

The 2021-03 training does not cover the use of these files.  You are free to examine them yourselves.

<summary>Example SPEC data file from a bluesky session:</summary>

<details>

    #F /home/mintadmin/Documents/projects/BCDA-APS/bluesky_training/20210723-205451.dat
    #E 1627091691
    #D Fri Jul 23 20:54:51 2021
    #C Bluesky  user = mintadmin  host = mint-vm
    #O0 
    #o0 

    #S 344  scan(detectors=['noisy', 'th_tth_permit'], num=11, args='['theta', 4.5, 3.0, 'ttheta', 9, 6]', per_step='None')
    #D Fri Jul 23 20:55:57 2021
    #C Fri Jul 23 20:55:57 2021.  plan_type = generator
    #C Fri Jul 23 20:55:57 2021.  uid = 17ebe732-8cff-458e-8ebd-f7c5207b2e46
    #MD uid = 17ebe732-8cff-458e-8ebd-f7c5207b2e46
    #MD beamline_id = Bluesky_training
    #MD detectors = ['noisy', 'th_tth_permit']
    #MD instrument_name = Bluesky Case Studies
    #MD login_id = mintadmin@mint-vm
    #MD motors = ('theta', 'ttheta')
    #MD notebook = UB_autosave
    #MD num_intervals = 10
    #MD num_points = 11
    #MD objective = Demonstrate UB matrix save & restore
    #MD pid = 10389
    #MD plan_pattern = inner_product
    #MD plan_pattern_args = {'num': 11, 'args': ["EpicsMotor(prefix='gp:m1', name='theta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 4.5, 3.0, "EpicsMotor(prefix='gp:m2', name='ttheta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 9, 6]}
    #MD plan_pattern_module = bluesky.plan_patterns
    #MD proposal_id = training
    #MD versions = {'apstools': '1.5.1', 'bluesky': '1.7.0', 'databroker': '1.2.3', 'epics': '3.5.0', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.3', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 8
    #L theta  ttheta  Epoch_float  Epoch  ttheta_user_setpoint  theta_user_setpoint  noisy  th_tth_permit
    4.5 9.0 9.998106479644775 10 9.0 4.5 0.0 31963.763745860513
    4.3500000000000005 8.700000000000001 10.893603086471558 11 8.7 4.35 0.0 34611.37532330056
    4.2 8.4 11.595271110534668 12 8.4 4.2 0.0 35180.93342784417
    4.05 8.1 12.299082040786743 12 8.1 4.05 0.0 37694.92329152548
    3.9 7.8 13.00736141204834 13 7.8 3.9 0.0 39442.76049515945
    3.75 7.5 13.71463131904602 14 7.5 3.75 0.0 42096.21324791565
    3.6 7.2 14.41884994506836 14 7.2 3.6 0.0 44406.552389513294
    3.45 6.9 15.12493348121643 15 6.9 3.45 0.0 45977.709130280506
    3.3000000000000003 6.6000000000000005 15.830729246139526 16 6.6 3.3 0.0 48881.20001936124
    3.15 6.3 16.53253173828125 17 6.300000000000001 3.1500000000000004 0.0 51082.06178377704
    3.0 6.0 17.24329447746277 17 6.0 3.0 0.0 53210.29908657747
    #C Fri Jul 23 20:56:14 2021.  num_events_baseline = 2
    #C Fri Jul 23 20:56:14 2021.  num_events_primary = 11
    #C Fri Jul 23 20:56:14 2021.  exit_status = success

    #S 345  scan(detectors=['noisy', 'th_tth_permit'], num=4, args='['theta', 3.5, 3.0, 'ttheta', 7, 6]', per_step='None')
    #D Fri Jul 23 20:57:50 2021
    #C Fri Jul 23 20:57:50 2021.  plan_type = generator
    #C Fri Jul 23 20:57:50 2021.  uid = 5761047b-ac05-428a-8b9c-6601b0aba2c0
    #MD uid = 5761047b-ac05-428a-8b9c-6601b0aba2c0
    #MD beamline_id = Bluesky_training
    #MD detectors = ['noisy', 'th_tth_permit']
    #MD instrument_name = Bluesky Case Studies
    #MD login_id = mintadmin@mint-vm
    #MD motors = ('theta', 'ttheta')
    #MD notebook = UB_autosave
    #MD num_intervals = 3
    #MD num_points = 4
    #MD objective = Demonstrate UB matrix save & restore
    #MD pid = 10389
    #MD plan_pattern = inner_product
    #MD plan_pattern_args = {'num': 4, 'args': ["MyMotor(prefix='gp:m1', name='theta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 3.5, 3.0, "MyMotor(prefix='gp:m2', name='ttheta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 7, 6]}
    #MD plan_pattern_module = bluesky.plan_patterns
    #MD proposal_id = training
    #MD versions = {'apstools': '1.5.1', 'bluesky': '1.7.0', 'databroker': '1.2.3', 'epics': '3.5.0', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.3', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 12
    #L theta  ttheta  Epoch_float  Epoch  theta_user_setpoint  theta_backlash  theta_backlash_velocity  ttheta_user_setpoint  ttheta_backlash  ttheta_backlash_velocity  noisy  th_tth_permit
    3.5 7.0 4.44653844833374 4 3.5 0.0 1.0 7.0 0.5 0.2 0.0 46002.45836582881
    3.333 6.67 8.672361612319946 9 3.3333333333333335 0.0 1.0 6.666666666666667 0.5 0.2 0.0 46875.28455439954
    3.1670000000000003 6.33 12.796637535095215 13 3.1666666666666665 0.0 1.0 6.333333333333333 0.5 0.2 0.0 50480.82899166115
    3.0 6.0 16.955684661865234 17 3.0 0.0 1.0 6.0 0.5 0.2 0.0 54033.337551435354
    #C Fri Jul 23 20:58:08 2021.  num_events_baseline = 2
    #C Fri Jul 23 20:58:08 2021.  num_events_primary = 4
    #C Fri Jul 23 20:58:08 2021.  exit_status = success

    #S 346  scan(detectors=['noisy', 'th_tth_permit'], num=4, args='['theta', 4.0, 3.0, 'ttheta', 8, 6]', per_step='None')
    #D Fri Jul 23 20:58:28 2021
    #C Fri Jul 23 20:58:28 2021.  plan_type = generator
    #C Fri Jul 23 20:58:28 2021.  uid = 14e4398d-42b5-40cf-b082-e2f7e4fd07fd
    #MD uid = 14e4398d-42b5-40cf-b082-e2f7e4fd07fd
    #MD beamline_id = Bluesky_training
    #MD detectors = ['noisy', 'th_tth_permit']
    #MD instrument_name = Bluesky Case Studies
    #MD login_id = mintadmin@mint-vm
    #MD motors = ('theta', 'ttheta')
    #MD notebook = UB_autosave
    #MD num_intervals = 3
    #MD num_points = 4
    #MD objective = Demonstrate UB matrix save & restore
    #MD pid = 10389
    #MD plan_pattern = inner_product
    #MD plan_pattern_args = {'num': 4, 'args': ["MyMotor(prefix='gp:m1', name='theta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 4.0, 3.0, "MyMotor(prefix='gp:m2', name='ttheta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 8, 6]}
    #MD plan_pattern_module = bluesky.plan_patterns
    #MD proposal_id = training
    #MD versions = {'apstools': '1.5.1', 'bluesky': '1.7.0', 'databroker': '1.2.3', 'epics': '3.5.0', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.3', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 12
    #L theta  ttheta  Epoch_float  Epoch  theta_user_setpoint  theta_backlash  theta_backlash_velocity  ttheta_user_setpoint  ttheta_backlash  ttheta_backlash_velocity  noisy  th_tth_permit
    4.0 8.0 5.639213562011719 6 4.0 0.0 1.0 8.0 0.5 0.2 0.0 37946.01826904763
    3.6670000000000003 7.33 10.168693780899048 10 3.6666666666666665 0.0 1.0 7.333333333333333 0.5 0.2 0.0 43240.06624621665
    3.333 6.67 14.600727081298828 15 3.3333333333333335 0.0 1.0 6.666666666666667 0.5 0.2 0.0 47339.97568055583
    3.0 6.0 19.046493530273438 19 3.0 0.0 1.0 6.0 0.5 0.2 0.0 52702.49931930254
    #C Fri Jul 23 20:58:48 2021.  num_events_baseline = 2
    #C Fri Jul 23 20:58:48 2021.  num_events_primary = 4
    #C Fri Jul 23 20:58:48 2021.  exit_status = success

    #S 347  scan(detectors=['noisy', 'th_tth_permit'], num=4, args='['theta', 4.0, 3.0, 'ttheta', 8, 6]', per_step='None')
    #D Fri Jul 23 21:03:30 2021
    #C Fri Jul 23 21:03:30 2021.  plan_type = generator
    #C Fri Jul 23 21:03:30 2021.  uid = 8869043f-56fc-4b3b-ad75-bc024694e6d6
    #MD uid = 8869043f-56fc-4b3b-ad75-bc024694e6d6
    #MD beamline_id = Bluesky_training
    #MD detectors = ['noisy', 'th_tth_permit']
    #MD instrument_name = Bluesky Case Studies
    #MD login_id = mintadmin@mint-vm
    #MD motors = ('theta', 'ttheta')
    #MD notebook = UB_autosave
    #MD num_intervals = 3
    #MD num_points = 4
    #MD objective = Demonstrate UB matrix save & restore
    #MD pid = 10389
    #MD plan_pattern = inner_product
    #MD plan_pattern_args = {'num': 4, 'args': ["MyMotor(prefix='gp:m1', name='theta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 4.0, 3.0, "MyMotor(prefix='gp:m2', name='ttheta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 8, 6]}
    #MD plan_pattern_module = bluesky.plan_patterns
    #MD proposal_id = training
    #MD versions = {'apstools': '1.5.1', 'bluesky': '1.7.0', 'databroker': '1.2.3', 'epics': '3.5.0', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.3', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 12
    #L theta  ttheta  Epoch_float  Epoch  theta_user_setpoint  theta_backlash  theta_backlash_velocity  ttheta_user_setpoint  ttheta_backlash  ttheta_backlash_velocity  noisy  th_tth_permit
    4.0 8.0 5.540862798690796 6 4.0 0.0 1.0 8.0 0.5 0.2 0.0 37996.6450939875
    3.6670000000000003 7.33 9.968387842178345 10 3.6666666666666665 0.0 1.0 7.333333333333333 0.5 0.2 0.0 42649.32391462264
    3.333 6.67 14.498717546463013 14 3.3333333333333335 0.0 1.0 6.666666666666667 0.5 0.2 0.0 46995.88551308093
    3.0 6.0 19.0245144367218 19 3.0 0.0 1.0 6.0 0.5 0.2 0.0 54150.34088276983
    #C Fri Jul 23 21:03:49 2021.  num_events_baseline = 2
    #C Fri Jul 23 21:03:49 2021.  num_events_primary = 4
    #C Fri Jul 23 21:03:49 2021.  exit_status = success

    #S 348  scan(detectors=['noisy', 'th_tth_permit'], num=4, args='['theta', 4.0, 3.0, 'ttheta', 8, 6]', per_step='None')
    #D Fri Jul 23 21:04:04 2021
    #C Fri Jul 23 21:04:04 2021.  plan_type = generator
    #C Fri Jul 23 21:04:04 2021.  uid = e9498d10-8fb7-41c4-aaba-f5204db32761
    #MD uid = e9498d10-8fb7-41c4-aaba-f5204db32761
    #MD beamline_id = Bluesky_training
    #MD detectors = ['noisy', 'th_tth_permit']
    #MD instrument_name = Bluesky Case Studies
    #MD login_id = mintadmin@mint-vm
    #MD motors = ('theta', 'ttheta')
    #MD notebook = UB_autosave
    #MD num_intervals = 3
    #MD num_points = 4
    #MD objective = Demonstrate UB matrix save & restore
    #MD pid = 10389
    #MD plan_pattern = inner_product
    #MD plan_pattern_args = {'num': 4, 'args': ["MyMotor(prefix='gp:m1', name='theta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 4.0, 3.0, "MyMotor(prefix='gp:m2', name='ttheta', settle_time=0.0, timeout=None, read_attrs=['user_readback', 'user_setpoint', 'backlash', 'backlash_velocity'], configuration_attrs=['user_offset', 'user_offset_dir', 'velocity', 'acceleration', 'motor_egu'])", 8, 6]}
    #MD plan_pattern_module = bluesky.plan_patterns
    #MD proposal_id = training
    #MD versions = {'apstools': '1.5.1', 'bluesky': '1.7.0', 'databroker': '1.2.3', 'epics': '3.5.0', 'h5py': '3.2.1', 'intake': '0.6.2', 'matplotlib': '3.3.4', 'numpy': '1.20.3', 'ophyd': '1.6.1', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 12
    #L theta  ttheta  Epoch_float  Epoch  theta_user_setpoint  theta_backlash  theta_backlash_velocity  ttheta_user_setpoint  ttheta_backlash  ttheta_backlash_velocity  noisy  th_tth_permit
    4.0 8.0 5.37143349647522 5 4.0 0.0 1.0 8.0 0.5 0.2 0.0 38949.839143231766
    3.6670000000000003 7.33 9.859706163406372 10 3.6666666666666665 0.0 1.0 7.333333333333333 0.5 0.2 0.0 43461.79736667226
    3.333 6.67 14.294379949569702 14 3.3333333333333335 0.0 1.0 6.666666666666667 0.5 0.2 0.0 47545.930496638146
    3.0 6.0 18.838080167770386 19 3.0 0.0 1.0 6.0 0.5 0.2 0.0 53464.77967307891
    #C Fri Jul 23 21:04:23 2021.  num_events_baseline = 2
    #C Fri Jul 23 21:04:23 2021.  num_events_primary = 4
    #C Fri Jul 23 21:04:23 2021.  exit_status = success

</details>

## User Code File

A common request from instruments is to allow the user to write some python code that can be loaded into the current session.  Since this codedevelopment of this *user code* is often iterative, it must be possible to reload the code without requiring the session to exit and restart.

The local file [`pv_finder.py`](pv_finder.py) provides an example of such a user code file.  Load this file with the command:

    %run -im pv_finder

Note that since the `m` (as module) command option was added, it is not necessary to give the Python extension `.py`.

This file loads the command `findpv(PVNAME)` which identifies the name of the ophyd object(s) using the PV name passed as an argument.  Take for example, we want to find PV `gp:userCalc8.CALC`:

In [13]:
%run -im pv_finder

In [15]:
findpv("gp:userCalc8.CALC")

{'read': ['calcs.calc8.calculation', 'temperature.calculation'],
 'write': ['calcs.calc8.calculation', 'temperature.calculation']}

Observe that the same `.calculation` is both readable and writable from two different ophyd objects: `calcs.calc8` and `temperature`.

## databroker

The [`databroker`](https://blueskyproject.io/databroker) package provides 
a Python interface to the database with the experiment data, including 
references to the large file content such as area detector images.  A 
[YAML](https://yaml.org) configuration file connects databroker with a 
specific repository in the MongoDB database server.  In the example here 
([`bluesky_class.yml`](./bluesky_class.yml)), the name of the *catalog* entry
in this file is `class_2021_03`.  It makes two connections to a MongoDB server
running on the same workstation `localhost`.  Both connections are to the 
same MongoDB collection: `class_2021_03-bluesky`.  The name of the file
is not important as long as it is placed in a directory searched by 
`databroker.catalog`.

**Example**

```yml
# file: bluesky_class.yml
# purpose: Configuration file to connect Bluesky databroker with MongoDB
# For 2021-03 Python Training at APS

# Copy to: ~/.local/share/intake/bluesky_class.yml
# Create subdirectories as needed

sources:
  class_2021_03:
    args:
      asset_registry_db: mongodb://localhost:27017/class_2021_03-bluesky
      metadatastore_db: mongodb://localhost:27017/class_2021_03-bluesky
    driver: bluesky-mongo-normalized-catalog
```

When the bluesky session starts, this `class_2021_03` catalog is referenced when
creating the `db` object in the instrument package, in 
`instrument.framework.initialize.py` by these lines:

```python
catalog_name = "class_2021_03"
db = databroker.catalog[catalog_name].v1
logger.info(f"using databroker catalog '{catalog_name}'")
```